In [45]:
import numpy as np; 
import pandas as pd

Step 3. Construct a classifier
Boosting Algorithms and Importance of XGBoosting 
The name xgboost, though, actually refers to the engineering goal to push the limit of computations resources for boosted tree algorithms. Which is the reason why many people use xgboost.

Gradient Boosting algorithm also called gradient boosting machine including the learning rate.
Stochastic Gradient Boosting with sub-sampling at the row, column and column per split levels.
Regularized Gradient Boosting with both L1 and L2 regularization.

The two reasons to use XGBoost are also the two goals of the project:

1.Execution Speed.
2.Model Performance.

The XGBoost library implements the gradient boosting decision tree algorithm

Link:-https://en.wikipedia.org/wiki/Gradient_boosting


In [46]:

# Load manipulated data set
train_df = pd.read_pickle("train_set.csv")
test_df  = pd.read_pickle("test_set.csv")

In [47]:
# Function to make labels in the data frame into a list (i.e. 0 8 => [0, 8])
def labels_to_list(labels): return list(map(int, labels.split()))

# Process train & test set into an array format
X_train = np.array([x for x in train_df['features']])
Y_train = np.array([labels_to_list(y) for y in train_df['labels']])
X_test = np.array([x for x in test_df['features']])

# Check shape of array-format train & test set
print("X_train: ", X_train.shape)
print("Y_train: ", Y_train.shape)
print("X_test: ", X_test.shape)

X_train:  (1996, 2048)
Y_train:  (1996,)
X_test:  (10000, 2048)


Labeling is the process of encoding Different categories.

We have main two approchs for labeling categorical data 1.LabelEncoder for encoding two label data like M/F Distiguation 2.one hot encoding is process of encoding label with considering dependency there is no correlation between previous labels and present labels. In this problem we need one-hot encoding for the multilable classificatin problem so we are using Multilabelbinarizer for encoding the categorical data.

Advantages and disadvantages of encoding:

1.Label encoding will not enhance dimensionality of dataset 2.But By using one-hot encoding we are getting high dimensionality of dataset but we can do dimensionality reduction using PCA to select principle components of high variance features so that computational time will reduce for training the model.


In [48]:
# Load packages for splitting train & validation set, XGBoost classifier, 1-of-K encoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier

# Package for estimating a time taken
import time; t=time.time()

Model Traning Process:

1.Traning is important process of Machine learning phase its important to handle the overfitting and underfitting when we training the model using classifier. splitting the dataset to train and test splits is important we have different splitting methodoligies for overcomming overfitting and under fitting probelm Ex:- K-fold cross validation and train_test_split are commonly used methods for overcomming overfitting fitting problem.


In [49]:
# Convert list of labels to follow 1-of-K coding scheme
one_of_K_encoder = MultiLabelBinarizer()
Y_train_ = one_of_K_encoder.fit_transform(Y_train)

# Split train set into 8:2 (train : validation)
random_state = np.random.RandomState(1040941203)
X_train_, X_test_, Y_train_, Y_test_ = train_test_split(X_train, Y_train_, test_size=0.2, random_state=random_state)

# Train the XGBoost classifier
classifier = OneVsRestClassifier(XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6, nthread=4, n_estimators=1000, objective="multi:softmax"))
classifier.fit(X_train_, Y_train_)

# Predict labels using the trained model
Y_predict = classifier.predict(X_test_)

# Show spent time
print("Time passed: ", "{0:.3f}".format(time.time() - t), "sec")

Time passed:  4373.469 sec


In [50]:
# Show some predicted values
print("Samples of predicted labels (in 1-of-K coding scheme):\n", Y_predict[1:3])
print("\nSamples of corresponding predicted labels:\n", one_of_K_encoder.inverse_transform(Y_predict[1:3]))

Samples of predicted labels (in binary matrix):
 [[0 1 1 0 0 1 1 0 0]
 [0 0 0 0 0 0 1 0 1]]

Samples of predicted labels:
 [(1, 2, 5, 6), (6, 8)]


Once we done labeling and model classification its good to test the classifier accuracy

First we can see percentage of each lable prediction in prediction dataset


In [51]:
# Construct a data frame to show ratio of each label in a predicted set
stat = pd.DataFrame(columns = ['label ' + str(i) for i in range(9)] + ['total_biz'], index = ['biz_count', 'biz_percentage'])

stat.loc['biz_count'] = np.append(np.sum(Y_predict, axis=0), len(Y_predict))
stat.loc['biz_percentage'] = stat.loc['biz_count'] * 100 / len(Y_predict) 

pd.options.display.float_format = '{:.0f}%'.format

# Show the statistics
stat

,label 0,label 1,label 2,label 3,label 4,label 5,label 6,label 7,label 8,total_biz
biz_count,103,202,221,207,114,264,287,121,236,400
biz_percentage,26%,50%,55%,52%,28%,66%,72%,30%,59%,100%


The F1 Score is the 2((precision ) (recall)/(precision+recall)). Link for reference: https://en.wikipedia.org/wiki/F1_score Importance of using F1 Score precision:- Precision is the number of True Positives divided by the number of True Positives and False Positives Recall: Recall is the number of True Positives divided by the number of True Positives and the number of False Negatives. 

In [52]:
from sklearn.metrics import f1_score # For measuring F1 score metrics

# Show global F1 score & on-label F1 score
print("Overall F1 score: ", f1_score(Y_test_, Y_predict, average='micro')) 
print("F1 score of each label : ", f1_score(Y_test, y_ppredict, average=None))

F1 score:  0.8349900596421471
Individual Class F1 score:  [0.66666667 0.83538084 0.88735632 0.66666667 0.77118644 0.88593156
 0.95017794 0.79518072 0.875     ]


In [53]:
t = time.time()

# Convert list of labels to follow 1-of-K coding scheme
one_of_K_encoder = MultiLabelBinarizer()
Y_train_ = one_of_K_encoder.fit_transform(Y_train)

# Train the SVM classifier again with a full train set
random_state = np.random.RandomState(0)
classifier = OneVsRestClassifier(XGBClassifier(num_class=9, gamma=0.024, learning_rate=0.3, max_depth=6, nthread=4, n_estimators=1000, objective="multi:softmax"))
classifier.fit(X_train, Y_train_)

Y_predict = classifier.predict(X_test)
Y_predict_label = one_of_K_encoder.inverse_transform(Y_predict)

print("Time passed: ", "{0:.1f}".format(time.time() - t), "sec")

Time passed:  5864.1 sec


After Doing classification Analysis and accuracy measure we can transform the one-hot prediction labels to normal labeling form we can use inverse_transfrom of predicted label for total submittion.

In [54]:
# Construct a data frame to show ratio of each label in a predicted set
stat = pd.DataFrame(columns=['label ' + str(i) for i in range(9)] + ['total_biz'], index = ['biz_count', 'biz_percentage'])

stat.loc['biz_count'] = np.append(np.sum(Y_predict, axis=0), len(Y_predict))
stat.loc['biz_percentage'] = stat.loc["biz_count"] * 100 / len(Y_predict)

pd.options.display.float_format = '{:.0f}%'.format

stat

,label 0,label 1,label 2,label 3,label 4,label 5,label 6,label 7,label 8,total_biz
biz_count,760,7663,8446,7482,1540,9084,9344,2097,6547,10000
biz_percentage,8%,77%,84%,75%,15%,91%,93%,21%,65%,100%


In [55]:
# Construct a data frame for submission (matching predicted label with business id in a test set)
final_df = pd.DataFrame(columns=['business_id','labels'])

for i in range(len(test_df)):
    biz = test_df.loc[i]['business_id']
    label = Y_predict_label[i]
    label = str(label)[1:-1].replace(",", " ")
    
    final_df.loc[i] = [str(biz), label]

# Write a submission file
with open("Harish_project_work.csv",'w') as file:
    final_df.to_csv(file, index=False)